In [ ]:
# Download needed packages 
!pip3 install -U vnstock3
!pip3 install pandas
!pip3 install openpyxl

In [ ]:
import pandas as pd
# Open Vnstock3 and accecpt default asking
from vnstock3 import Vnstock
import os
from google.colab import files

In [ ]:
# Initialize VN stock object from 'VCI' without giving any initial symbol 
stock = Vnstock().stock(source='VCI')


In [ ]:
def clear_data_from_file(name): 
    if os.path.exists(name):
        os.remove(name)


In [ ]:
def append_to_file(name, data):
    df_new = pd.DataFrame(data)
    
    try:
        df_combined = df_new
        # Save the combined data to Excel
        df_new.to_excel(name, index=False)
    
    except FileNotFoundError:
        # If file does not exist, create a new one
        df_new.to_excel(name, index=False)

In [ ]:
def get_filtered_prices(stock_df):
  ''''
  get price for first day of month. If the first day went on holidays, weekends, or does not have value, then get the valid day before.
  '''

  # Ensure the 'time' column is in datetime format
  stock_df['time'] = pd.to_datetime(stock_df['time'])
    
  # Set 'time' as the index
  stock_df.set_index('time', inplace=True)
    
  # Resample to monthly frequency, taking the first available entry
  stock_filtered_df = stock_df.resample('MS').asfreq().ffill()

  # Reset the index to bring 'time' back as a column (optional)
  stock_filtered_df.reset_index(inplace=True)

  return stock_filtered_df

In [ ]:
excel_file = 'demo.xlsx'

clear_data_from_file(excel_file)
# List all stocks from VN30
vn_stock_list = stock.listing.symbols_by_group('VN30')
main_df = pd.DataFrame()

for index in range(len(vn_stock_list)):
	# Stock price history from 15/04/2019 to 15/04/2024
	stock_history_df = stock.quote.history(symbol=vn_stock_list[index], start='2019-04-15', end='2024-04-15', interval='1D')
 
	# Filter out by first day of month
	stock_filtered_df = get_filtered_prices(stock_history_df)
	
	# Collect only time and close value 
	stock_selected_df = stock_history_df[['time', 'close']]
  
	df_to_excel = pd.DataFrame({
    	vn_stock_list[index]: stock_selected_df['close']
	})
	main_df = pd.concat([main_df, df_to_excel], axis=1)

print(main_df)

append_to_file(excel_file, main_df)

# Download the file in Colab
files.download(excel_file)
